In [2]:
import sqlite3
import os
import pandas as pd
import xml.etree.ElementTree as ET

"""results 파일에서 엑셀 변환"""
# ✅ Excel 파일이 있는 폴더 경로
folder_path = r"C:\VISSIM_Workspace\network_final"
file_path = r"C:\VISSIM_Workspace\network_final\network01.inpx"

from_time = 1800
siml_count = 10

# XML 파일 파싱
tree = ET.parse(file_path)
root = tree.getroot()

for elem in root.iter("dataColl"):
    interval_value = int(elem.get("interval"))  # "interval" 속성 값 가져오기

results_path = folder_path + "\\" + "network01.results"
#results_files = sorted([f for f in os.listdir(results_path) if f.endswith(".db")], key=lambda x: int(x.split(".")[0]))
results_files = ["1.db", "11.db"]
#print(results_files)

for results_file in results_files:
    # .db파일 데이터의 평균 도출
    db_path = os.path.join(results_path, results_file)
    # 데이터베이스 연결
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # 특정 테이블의 데이터 조회
    table_name = "DATACOLLECTIONMEASUREMENT_EvaluationTimeIntervalClass"
    query = f"SELECT * FROM {table_name};"  # 전체 데이터 조회

    # 데이터 읽기
    df = pd.read_sql_query(query, conn)

    # TIMEINT 변환
    df["ARG_TIMEINTERVAL"] = from_time + df["ARG_TIMEINTERVAL"].astype(int) * interval_value   # 변환 시작점 보정

    # OCCUPRATE 보정
    df["OCCUPRATE"] = df["OCCUPRATE"].astype(float)*100
    df["OCCUPRATE"] = df["OCCUPRATE"].apply(lambda x: f"{x:.2f}")  # 항상 소수점 둘째 자리까지 유지
    df = df.round(2)
    df["OCCUPRATE"] = df["OCCUPRATE"].astype(str) + "%"

    # SIMRUN 설정
    df["SIMRUN"] =siml_count
    df["OBJECT_ID"] = df["OBJECT_ID"].astype(int)
    # 컬럼명 변경
    df.rename(columns={
        "ARG_TIMEINTERVAL": "TIMEINT",
        "OBJECT_ID": "DATACOLLECTIONMEASUREMENT",
        "ACCELERATION" : "ACCELERATION(ALL)",
        "DIST" : "DIST(ALL)",
        "LENGTH" : "LENGTH(ALL)",
        "VEHS" : "VEHS(ALL)",
        "PERS" : "PERS(ALL)",
        "QUEUEDELAY" : "QUEUEDELAY(ALL)",
        "SPEEDAVGARITH" : "SPEEDAVGARITH(ALL)",
        "SPEEDAVGHARM" : "SPEEDAVGHARM(ALL)",
        "OCCUPRATE" : "OCCUPRATE(ALL)"
    }, inplace=True)

    # 순서 재정의
    df = df[["SIMRUN", "TIMEINT", "DATACOLLECTIONMEASUREMENT", "ACCELERATION(ALL)", "DIST(ALL)",
             "LENGTH(ALL)", "VEHS(ALL)", "PERS(ALL)", "QUEUEDELAY(ALL)", "SPEEDAVGARITH(ALL)", "SPEEDAVGHARM(ALL)", "OCCUPRATE(ALL)"]]

    excel_dir  = os.path.join(folder_path, "network02_output")
    if not os.path.exists(excel_dir):
        os.makedirs(excel_dir)
    last_file = xlsx_files = sorted([f for f in os.listdir(excel_dir) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)

    if not last_file:
        last_file = 1
    else:
        last_file = int(xlsx_files[-1].split("_")[0]) + 1  # 가장 마지막 파일 번호 + 1
    excel_path = os.path.join(excel_dir, f"{last_file}_output.xlsx")
    df.to_excel(excel_path, index=False)
    print(f"생성파일: {excel_path}")

생성파일: C:\VISSIM_Workspace\network_final\network02_output\1_output.xlsx
생성파일: C:\VISSIM_Workspace\network_final\network02_output\2_output.xlsx
